In [1]:
import pandas as pd
import numpy as np

In [2]:
plays = pd.read_csv('../data/tracking_week_2.csv')

# Grab only items that are before and during the snap
plays = plays[plays['frameType'].isin(['BEFORE_SNAP', 'BALL_SNAP'])]

# drop rows that are about the football
plays = plays[plays['displayName'] != 'football']

In [3]:
# Find the player with the maximum 's' within each 'gameId' and 'playId'
max_s_players = plays.loc[plays.groupby(['gameId', 'playId'])['s'].idxmax()]
# Get the row before the maximum 's' within each 'gameId' and 'playId'
previous_rows = plays.loc[plays.index.isin(max_s_players.index - 1)]
# Combine the rows
max_s_players = pd.concat([previous_rows, max_s_players]).sort_index()

max_s_players.head()

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
2820,2022091901,64,47836.0,Miles Sanders,16,BEFORE_SNAP,2022-09-20 00:32:04.8,26.0,PHI,right,22.27,36.11,3.68,0.62,0.37,23.32,17.37,NaN
2821,2022091901,64,47836.0,Miles Sanders,17,BEFORE_SNAP,2022-09-20 00:32:04.9,26.0,PHI,right,22.39,36.47,3.72,0.51,0.37,24.22,18.29,NaN
4562,2022091901,112,38559.0,Harrison Smith,2,BEFORE_SNAP,2022-09-20 00:33:10.8,22.0,MIN,right,61.02,16.52,1.18,0.35,0.12,14.06,18.91,NaN
4563,2022091901,112,38559.0,Harrison Smith,3,BEFORE_SNAP,2022-09-20 00:33:10.9,22.0,MIN,right,61.06,16.64,1.20,0.36,0.12,11.99,17.08,NaN
7609,2022091901,133,42388.0,Eric Kendricks,3,BEFORE_SNAP,2022-09-20 00:33:42.4,54.0,MIN,right,67.30,26.55,1.21,0.37,0.12,243.53,214.90,NaN


In [4]:
max_s_players['orientation_sin'] = np.sin(np.radians(max_s_players['o']))
max_s_players['orientation_cos'] = np.cos(np.radians(max_s_players['o']))
max_s_players['forward_velocity'] = max_s_players['s'] * max_s_players['orientation_cos']
max_s_players['sideways_velocity'] = max_s_players['s'] * max_s_players['orientation_sin']

In [5]:
def determine_direction(df):
    # Ensure the dataframe is sorted by gameId, playId, and frameId
    df = df.sort_values(by=['gameId', 'playId', 'frameId'])
    
    # Calculate the difference in y values between consecutive frames
    df['y_diff'] = df.groupby(['gameId', 'playId'])['y'].diff()
    
    # Define the conditions for the new column
    conditions = [
        (df['o'] > 180) & (df['y_diff'] > 0),
        (df['o'] > 180) & (df['y_diff'] < 0),
        (df['o'] < 180) & (df['y_diff'] > 0),
        (df['o'] < 180) & (df['y_diff'] < 0)
    ]
    
    # Define the corresponding values for the new column
    choices = ['right', 'left', 'left', 'right']
    
    # Create the new column based on the conditions
    df['direction'] = np.select(conditions, choices, default=np.nan)
    
    return df

# Apply the function to the max_s_players dataframe
max_s_players = determine_direction(max_s_players)
max_s_players.head()

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,...,dis,o,dir,event,orientation_sin,orientation_cos,forward_velocity,sideways_velocity,y_diff,direction
6255788,2022091500,55,47839.0,Mecole Hardman,119,BEFORE_SNAP,2022-09-16 00:16:33,17.0,KC,left,...,0.70,333.26,3.48,NaN,-0.449943,0.893057,6.331778,-3.190093,NaN,nan
6255789,2022091500,55,47839.0,Mecole Hardman,120,BEFORE_SNAP,2022-09-16 00:16:33.1,17.0,KC,left,...,0.72,332.53,2.31,NaN,-0.461284,0.887252,6.459198,-3.358148,0.72,right
6258507,2022091500,76,46243.0,Marquez Valdes-Scantling,15,BEFORE_SNAP,2022-09-16 00:16:54.4,11.0,KC,left,...,0.37,229.72,210.40,NaN,-0.762894,-0.646524,-2.405067,-2.837966,NaN,nan
6258508,2022091500,76,46243.0,Marquez Valdes-Scantling,16,BEFORE_SNAP,2022-09-16 00:16:54.5,11.0,KC,left,...,0.37,232.81,211.03,NaN,-0.796635,-0.604460,-2.266725,-2.987383,-0.32,left
6261867,2022091500,97,46213.0,Justin Watson,75,BEFORE_SNAP,2022-09-16 00:17:39.7,84.0,KC,left,...,0.62,177.16,178.58,NaN,0.049547,-0.998772,-6.232336,0.309174,NaN,nan


In [6]:
max_s_players = max_s_players.dropna(subset=['y_diff'])
max_s_players.head()

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,...,dis,o,dir,event,orientation_sin,orientation_cos,forward_velocity,sideways_velocity,y_diff,direction
6255789,2022091500,55,47839.0,Mecole Hardman,120,BEFORE_SNAP,2022-09-16 00:16:33.1,17.0,KC,left,...,0.72,332.53,2.31,NaN,-0.461284,0.887252,6.459198,-3.358148,0.72,right
6258508,2022091500,76,46243.0,Marquez Valdes-Scantling,16,BEFORE_SNAP,2022-09-16 00:16:54.5,11.0,KC,left,...,0.37,232.81,211.03,NaN,-0.796635,-0.604460,-2.266725,-2.987383,-0.32,left
6261868,2022091500,97,46213.0,Justin Watson,76,BEFORE_SNAP,2022-09-16 00:17:39.8,84.0,KC,left,...,0.63,177.16,178.33,NaN,0.049547,-0.998772,-6.272287,0.311156,-0.63,right
6266799,2022091500,118,53476.0,Asante Samuel,30,BEFORE_SNAP,2022-09-16 00:18:06.1,26.0,LAC,left,...,0.46,12.45,342.65,NaN,0.215588,0.976485,4.491829,0.991703,0.44,left
6272432,2022091500,171,44874.0,JuJu Smith-Schuster,19,BEFORE_SNAP,2022-09-16 00:19:22.2,9.0,KC,left,...,0.40,308.00,332.98,NaN,-0.788011,0.615661,2.438019,-3.120523,0.36,right


In [7]:
max_s_players['play_goes_right'] = pd.get_dummies(max_s_players['playDirection'])['right']
max_s_players.drop(columns=['playDirection'], inplace=True)
max_s_players.head()

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,x,...,o,dir,event,orientation_sin,orientation_cos,forward_velocity,sideways_velocity,y_diff,direction,play_goes_right
6255789,2022091500,55,47839.0,Mecole Hardman,120,BEFORE_SNAP,2022-09-16 00:16:33.1,17.0,KC,88.28,...,332.53,2.31,NaN,-0.461284,0.887252,6.459198,-3.358148,0.72,right,False
6258508,2022091500,76,46243.0,Marquez Valdes-Scantling,16,BEFORE_SNAP,2022-09-16 00:16:54.5,11.0,KC,85.06,...,232.81,211.03,NaN,-0.796635,-0.604460,-2.266725,-2.987383,-0.32,left,False
6261868,2022091500,97,46213.0,Justin Watson,76,BEFORE_SNAP,2022-09-16 00:17:39.8,84.0,KC,77.81,...,177.16,178.33,NaN,0.049547,-0.998772,-6.272287,0.311156,-0.63,right,False
6266799,2022091500,118,53476.0,Asante Samuel,30,BEFORE_SNAP,2022-09-16 00:18:06.1,26.0,LAC,65.88,...,12.45,342.65,NaN,0.215588,0.976485,4.491829,0.991703,0.44,left,False
6272432,2022091500,171,44874.0,JuJu Smith-Schuster,19,BEFORE_SNAP,2022-09-16 00:19:22.2,9.0,KC,64.38,...,308.00,332.98,NaN,-0.788011,0.615661,2.438019,-3.120523,0.36,right,False


In [10]:
# Select play here
max_s_players = max_s_players[max_s_players['gameId'] == 2022091100]
max_s_players = max_s_players[max_s_players['playId'] == 546]

max_s_players.head()

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,x,...,o,dir,event,orientation_sin,orientation_cos,forward_velocity,sideways_velocity,y_diff,direction,play_goes_right


In [8]:
'''df = max_s_players[['frameId', 'x', 'y', 's', 'a',
       'dis', 'o', 'dir', 'orientation_sin', 'orientation_cos',
       'forward_velocity', 'sideways_velocity', 'y_diff', 'play_goes_right']]'''

"df = max_s_players[['frameId', 'x', 'y', 's', 'a',\n       'dis', 'o', 'dir', 'orientation_sin', 'orientation_cos',\n       'forward_velocity', 'sideways_velocity', 'y_diff', 'play_goes_right']]"